In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets, transforms, utils
from dataset import datasetClass
from pathlib import Path
from settingReader import settingReader


In [3]:
import torchvision.models as models
resnet18 = models.resnet18(pretrained=True)
alexnet = models.alexnet(pretrained=True)

resnext50_32x4d = models.resnext50_32x4d(pretrained=True)


# Do not forget to enable GPUs!

In [4]:
def use_gpu():
    torch.set_default_tensor_type(torch.cuda.FloatTensor if torch.cuda.is_available() 
                                                         else torch.FloatTensor)
use_gpu()

# You might want to use custom transforms

In [5]:
transform = transforms.Compose([transforms.ToTensor()])

# Download the dataset

In [6]:

# trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=False, transform=transform)

# testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=False, transform=transform)
trainset = datasetClass(root='./data', train=True,download=True, transform=transform)

testset = datasetClass(root='./data', train=True,download=True, transform=transform)
print(trainset.targets)

settingInfo = settingReader()
# classess = ('plane', 'car', 'bird', 'cat',
#            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
classess = settingInfo.getItem("classNames")

hello
Files already downloaded and verified
hello
Files already downloaded and verified
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [7]:
print(len(trainset))
print(len(testset))


110
110


In [8]:


def train(model, train_loader, optimizer, epoch):
    model.train()
    # problem in the following for loop
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    print('Train Epoch: {}, Loss: {:.3f}'.format(epoch, loss.item()))


def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            _, pred = output.max(1)
            test_loss += criterion(output, target).item()

            correct += pred.eq(target).sum().item()
            total += target.size(0)
    test_loss /= len(test_loader.dataset)

    print('Average loss: {:.3f}, Test Acc: {:.3f} ({}/{})'.format(test_loss, 100.*correct/total, correct, total))
    
    return test_loss


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layer = nn.Sequential(
            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(4,4), padding=1), #used to be 4 by 4
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(4,4), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 2
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(4,4), padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(4,4), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.05),

            # Conv Layer block 3
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(4,4), padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(4,4), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(2048, 1024), #used to be 18432, 3072
            nn.ReLU(),
            nn.Linear(1024, 512), #used to be 3072, 1024
            nn.ReLU(),
            nn.Linear(512, 256),  #used to be 1024, 512
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(256, 20)#usded to be 512,20
        )

    def forward(self, x):
        if torch.cuda.is_available():
           x = x.to(device="cuda")
        # conv layers
        x = self.conv_layer(x)
        x = x.view(x.size(0), -1)


        # fc layer
        x = self.fc_layer(x)

        return x
batch_size = settingInfo.getItem("batchSize")
test_batch_size = 1
epochs = settingInfo.getItem("epochs")
lr = 1e-3
criterion = nn.CrossEntropyLoss()

#training
print(Path.cwd())
path = (Path.cwd() / "path.pth").resolve() #Does not work all the time. May need a direct file path. For mac, something like path = Path("/Users/userName/Desktop/ML/path.pth")
print(os.path.isfile(path))
model = CNN()
if os.path.isfile(path):
    model.load_state_dict(torch.load(path))
    model.eval()
optimizer = optim.Adam(model.parameters(), lr=lr)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(testset, batch_size=test_batch_size, shuffle=False)
losses = []
for epoch in range(1, epochs + 1):
    train(model, train_loader, optimizer, epoch)
    losses.append(test(model, val_loader))

torch.save(model.state_dict(), path)


/Users/games/Desktop/ML Github
False
Train Epoch: 1, Loss: 0.672
Average loss: 1.592, Test Acc: 50.000 (55/110)
Train Epoch: 2, Loss: 0.725
Average loss: 0.595, Test Acc: 100.000 (110/110)
Train Epoch: 3, Loss: 0.259
Average loss: 0.248, Test Acc: 90.909 (100/110)
Train Epoch: 4, Loss: 0.132
Average loss: 0.663, Test Acc: 68.182 (75/110)
Train Epoch: 5, Loss: 0.117
Average loss: 0.233, Test Acc: 86.364 (95/110)
